In [1]:
import sqlite3
import csv
import pandas as pd
import os

In [2]:
conn = sqlite3.connect('suicide.sqlite')
cur = conn.cursor()

In [3]:
cur.execute('''DROP TABLE IF EXISTS countries''') 
cur.execute('''DROP TABLE IF EXISTS happiness''') 
cur.execute('''DROP TABLE IF EXISTS suicide_rate''') 
cur.execute('''DROP TABLE IF EXISTS gdp_per_capita''')

cur.execute('''
            CREATE TABLE suicide_rate(
                id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
                country_name TEXT,
                gender TEXT,
                age_band TEXT,
                suicide_no integer,
                suicide_rate float,
                FOREIGN KEY(country_name)REFERENCES countries(country_name)
                FOREIGN KEY(country_name)REFERENCES happiness(country_name)
                FOREIGN KEY(country_name)REFERENCES gdp_per_capita(country_name))
                ''')
cur.execute('''
           CREATE TABLE countries(
                country_name TEXT,
                population integer,
                net_migration float,
                infant_mortality float,
                death_rate float,
                climate integer,
                literacy,
                PRIMARY KEY(country_name))''')
cur.execute('''
            CREATE TABLE happiness(
                country_name TEXT,
                happiness_score float,
                PRIMARY KEY(country_name))''')

        
cur.execute('''
            CREATE TABLE gdp_per_capita(
                country_name TEXT,
                gdp_per_capita float,
                PRIMARY KEY(country_name))''')
conn.commit()

In [4]:
cotw = pd.read_csv('/Users/yanfeiwang/Desktop2/python/input_shee/countries_clean.csv')

#next(cotw, None)

for index, row in cotw.iterrows():
    try:
        cur.execute('''
                INSERT INTO countries(
                country_name,
                population,
                net_migration,
                infant_mortality,
                death_rate,
                climate,
                literacy
                ) VALUES (?,?,?,?,?,?,?)''',(row[0],row[2],row[6],row[7],row[16],row[14],row[9])
               )
    except sqlite3.Error as e:
        conn.commit()

In [5]:
happiness = pd.read_csv('/Users/yanfeiwang/Desktop2/python/input_shee/happiness_clean.csv')

for index, row in happiness.iterrows():
    try:
        cur.execute('''
                INSERT INTO happiness(
                country_name,
                happiness_score
                ) VALUES (?,?)''',(row[0],row[3])
               )
    except sqlite3.Error as e:
        conn.commit()


In [6]:
suicide_rate = pd.read_csv('/Users/yanfeiwang/Desktop2/python/input_shee/suicide_clean.csv')

for index, row in suicide_rate.iterrows():
    try:
        cur.execute('''
                INSERT INTO suicide_rate(
                country_name,            
                gender,
                age_band,
                suicide_no,
                suicide_rate    
                ) VALUES (?,?,?,?,?)''',(row[0],row[1],row[2],row[3],row[5])
               )
    except sqlite3.Error as e:
        conn.commit()


In [7]:
gdp_per_capita = pd.read_csv('/Users/yanfeiwang/Desktop2/python/input_shee/gdp_per_capita.csv')


for index, row in gdp_per_capita.iterrows():
    try:
        cur.execute('''
                INSERT INTO gdp_per_capita(
                country_name,
                gdp_per_capita   
                ) VALUES (?,?)''',(row[0],row[1])
               )
    except sqlite3.Error as e:
        conn.commit()

In [8]:
cur.execute('''DROP TABLE IF EXISTS suicide_data''') 
cur.execute('''CREATE TABLE suicide_data AS
                SELECT suicide_rate.country_name, suicide_rate.suicide_rate, suicide_rate.gender, suicide_rate.age_band, countries.population, countries.net_migration, countries.infant_mortality, countries.death_rate, countries.climate, countries.literacy, gdp_per_capita.gdp_per_capita, happiness.happiness_score
                FROM suicide_rate JOIN countries JOIN happiness JOIN gdp_per_capita ON suicide_rate.country_name = happiness.country_name AND suicide_rate.country_name = countries.country_name AND suicide_rate.country_name = gdp_per_capita.country_name
                ORDER BY suicide_rate.country_name
            ''')

conn.commit()

In [9]:
data_df = pd.read_sql_query("SELECT * FROM suicide_data", conn)

In [10]:
data_df['gender'].value_counts()

male      318
female    318
Name: gender, dtype: int64

In [11]:
data_df['age_band'].value_counts()

15-24 years    106
35-54 years    106
75+ years      106
25-34 years    106
5-14 years     106
55-74 years    106
Name: age_band, dtype: int64

In [12]:
data_df.head()

,country_name,suicide_rate,gender,age_band,population,net_migration,infant_mortality,death_rate,climate,literacy,gdp_per_capita,happiness_score
0,Argentina,34.38,male,75+ years,39921833,0.61,15.18,7.55,3.0,97.1,14981.0,6.574
1,Argentina,18.74,male,15-24 years,39921833,0.61,15.18,7.55,3.0,97.1,14981.0,6.574
2,Argentina,14.31,male,55-74 years,39921833,0.61,15.18,7.55,3.0,97.1,14981.0,6.574
3,Argentina,14.13,male,25-34 years,39921833,0.61,15.18,7.55,3.0,97.1,14981.0,6.574
4,Argentina,12.25,male,35-54 years,39921833,0.61,15.18,7.55,3.0,97.1,14981.0,6.574


In [13]:
type(data_df)

pandas.core.frame.DataFrame

In [14]:
data_df.loc[:, 'age_band'] = data_df['age_band'].str.replace(' years','')
data_df.loc[data_df['age_band'] == '5-14', 'age_band'] = '05-14'
data_df['age_band'] = data_df['age_band'].astype("category").cat.codes
data_df['gender'] = data_df['gender'].astype("category").cat.codes;

In [15]:
data_df.head(20)

,country_name,suicide_rate,gender,age_band,population,net_migration,infant_mortality,death_rate,climate,literacy,gdp_per_capita,happiness_score
0,Argentina,34.38,1,5,39921833,0.61,15.18,7.55,3.0,97.1,14981.0,6.574
1,Argentina,18.74,1,1,39921833,0.61,15.18,7.55,3.0,97.1,14981.0,6.574
2,Argentina,14.31,1,4,39921833,0.61,15.18,7.55,3.0,97.1,14981.0,6.574
3,Argentina,14.13,1,2,39921833,0.61,15.18,7.55,3.0,97.1,14981.0,6.574
4,Argentina,12.25,1,3,39921833,0.61,15.18,7.55,3.0,97.1,14981.0,6.574
5,Argentina,4.51,0,1,39921833,0.61,15.18,7.55,3.0,97.1,14981.0,6.574
6,Argentina,3.55,0,4,39921833,0.61,15.18,7.55,3.0,97.1,14981.0,6.574
7,Argentina,3.52,0,2,39921833,0.61,15.18,7.55,3.0,97.1,14981.0,6.574
8,Argentina,3.13,0,3,39921833,0.61,15.18,7.55,3.0,97.1,14981.0,6.574
9,Argentina,2.01,0,5,39921833,0.61,15.18,7.55,3.0,97.1,14981.0,6.574


In [16]:
suicide_df = data_df.drop('country_name', axis = 1);
suicide_df.to_csv('/Users/yanfeiwang/Desktop2/python/input_shee/suicide_df.csv', index=False)